In [1]:
from google.colab import files

uploaded = files.upload()

Saving NBAVideo_Data.zip to NBAVideo_Data (3).zip


In [2]:
import zipfile
import os
import pandas as pd
import glob

In [3]:
zip_file_name = 'NBAVideo_Data.zip'
extract_to_folder = 'NBAVideo_Data'

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_to_folder)

# List the files to verify extraction
extracted_files = os.listdir(extract_to_folder)
print(f"Extracted {len(extracted_files)} files.")

Extracted 132 files.


In [4]:
%config InlineBackend.figure_format='retina'
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%config InlineBackend.rc={'figure.figsize': (10.0, 6.0), 'font.size': 14}
%config InlineBackend.figure_formats = {'png', 'retina'}

%matplotlib inline

# Increase the rate limit
import IPython
from google.colab import output

display(IPython.display.HTML('''
  <script>
    google.colab.output.setIframeHeight(0, true, {maxHeight: 10000000000000});
  </script>
'''))

In [5]:
# Function to download videos
def download_videos(video_urls, output_directory):
    for i, url in enumerate(video_urls, start=1):
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'  # Add User-Agent header here
            }
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                with open(os.path.join(output_directory, f"video_{i}.mp4"), 'wb') as f:
                    f.write(response.content)
                print(f"Video {i} downloaded successfully.")
            else:
                print(f"Failed to download video {i}. Status code: {response.status_code}")
        except Exception as e:
            print(f"Error downloading video {i}: {e}")

In [8]:
pip install --upgrade yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.6 MB/s eta 0:00:00


In [11]:
import os
import pandas as pd
import glob

# Function to load all CSV files in a directory
def load_all_csv_files(directory_path):
    all_files = glob.glob(os.path.join(directory_path, "*.csv"))
    all_dataframes = []
    for file in all_files:
        df = pd.read_csv(file)
        all_dataframes.append(df)
    return all_dataframes

# Function to extract video URLs from each DataFrame
def extract_video_urls(dataframes):
    video_urls = []
    for df in dataframes:
        if 'video_url' in df.columns:
            urls = df['video_url'].tolist()
            # Split URLs separated by "</>" and add individual valid URLs
            for url in urls:
                if "</>" in url:
                    split_urls = url.split("</>")
                    video_urls.extend(split_urls)
                else:
                    video_urls.append(url)
    return video_urls

# Provide the path to the directory containing the CSV files
directory_path = 'NBAVideo_Data'

# Load all CSV files
dataframes = load_all_csv_files(directory_path)

# Extract video URLs
video_urls = extract_video_urls(dataframes)

# Display the extracted video URLs
for url in video_urls:
    print(url)

Streaming output truncated to the last 5000 lines.
 https://videos.nba.com/nba/pbp/media/2018/12/28/0021800526/226/a1d160bd-4006-d28d-756f-56332202b4a7_1280x720.mp4
https://videos.nba.com/nba/pbp/media/2018/12/28/0021800526/229/592aa1bf-de5f-a6c7-9f2c-050fd8747006_1280x720.mp4
https://videos.nba.com/nba/pbp/media/2018/12/28/0021800526/234/fe0edb90-3fc6-4cca-f437-ace7ed14c370_1280x720.mp4
https://videos.nba.com/nba/pbp/media/2018/12/28/0021800526/238/fd5fe35c-2ef8-069b-0198-0bb9223f8f65_1280x720.mp4 
 https://videos.nba.com/nba/pbp/media/2018/12/28/0021800526/239/17cc3e60-b343-283e-f48e-e6a00905f2dd_1280x720.mp4
https://videos.nba.com/nba/pbp/media/2018/12/28/0021800526/240/38a11c07-57be-44df-04ca-c672e77743c2_1280x720.mp4 
 https://videos.nba.com/nba/pbp/media/2018/12/28/0021800526/241/9c4dc47a-736e-b09d-8aeb-57f70828272f_1280x720.mp4
https://videos.nba.com/nba/pbp/media/2018/12/28/0021800526/242/9dba92d1-1780-fb60-8966-486e96134947_1280x720.mp4 
 https://videos.nba.com/nba/pbp/media/2

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
import subprocess

# Function to standardize videos using ffmpeg
def standardize_videos(input_directory, output_directory):
    # Get a list of all downloaded video files
    video_files = glob.glob(os.path.join(input_directory, '*.mp4'))

    # Standardize each video file
    for i, video_file in enumerate(video_files, start=1):
        try:
            # Define the output file path for the standardized video
            standardized_video_file = os.path.join(output_directory, f"standardized_video_{i}.mp4")

            # Use ffmpeg to convert the video to a standard format
            ffmpeg_cmd = ['ffmpeg', '-i', video_file, standardized_video_file]
            subprocess.run(ffmpeg_cmd, check=True)

            print(f"Video {i} standardized successfully.")
        except Exception as e:
            print(f"Error standardizing video {i}: {e}")

# Provide the input directory where the downloaded videos are located
input_directory = 'downloaded_videos'

# Provide the output directory where the standardized videos will be saved
output_directory = 'standardized_videos'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Standardize the downloaded videos
standardize_videos(input_directory, output_directory)

Error standardizing video 1: Command '['ffmpeg', '-i', 'downloaded_videos/video_96.mp4', 'standardized_videos/standardized_video_1.mp4']' returned non-zero exit status 1.
Error standardizing video 2: Command '['ffmpeg', '-i', 'downloaded_videos/video_97.mp4', 'standardized_videos/standardized_video_2.mp4']' returned non-zero exit status 1.
Error standardizing video 3: Command '['ffmpeg', '-i', 'downloaded_videos/video_88.mp4', 'standardized_videos/standardized_video_3.mp4']' returned non-zero exit status 1.
Error standardizing video 4: Command '['ffmpeg', '-i', 'downloaded_videos/video_30.mp4', 'standardized_videos/standardized_video_4.mp4']' returned non-zero exit status 1.
Error standardizing video 5: Command '['ffmpeg', '-i', 'downloaded_videos/video_60.mp4', 'standardized_videos/standardized_video_5.mp4']' returned non-zero exit status 1.
Error standardizing video 6: Command '['ffmpeg', '-i', 'downloaded_videos/video_116.mp4', 'standardized_videos/standardized_video_6.mp4']' return

KeyboardInterrupt: 

Video URLs saved to 'video_urls.txt'.
